In [1]:
# adopt rgb-d to read information
import cv2
import mediapipe as mp
import pyrealsense2 as rs
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# 创建一个管道对象
pipeline = rs.pipeline()

# 创建一个配置对象
config = rs.config()

# 配置RGB和深度流，设置最大的分辨率和帧率
config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 30) 

# 开始流
profile = pipeline.start(config)

align_to = rs.stream.color
align = rs.align(align_to)

with mp_hands.Hands(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
    try:
        while True:
            frames = pipeline.wait_for_frames()
            aligned_frames = align.process(frames)

            # 获取对齐后的深度和颜色帧
            aligned_depth_frame = aligned_frames.get_depth_frame()
            color_frame = aligned_frames.get_color_frame()

            if not aligned_depth_frame or not color_frame:
                continue

            color_image = np.asanyarray(color_frame.get_data())
            depth_image = np.asanyarray(aligned_depth_frame.get_data())

            # 将图像从 BGR 转换为 RGB
            color_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
            color_image.flags.writeable = False
            results = hands.process(color_image)

            color_image.flags.writeable = True
            color_image = cv2.cvtColor(color_image, cv2.COLOR_RGB2BGR)
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        color_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                    # 打印手腕坐标
                    wrist_landmark = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST]
                    wrist_x = min(int(wrist_landmark.x * color_image.shape[1]), color_image.shape[1] - 1)
                    wrist_y = min(int(wrist_landmark.y * color_image.shape[0]), color_image.shape[0] - 1)
                    wrist_z = depth_image[wrist_y, wrist_x].astype(float)
                    print("手腕坐标 (图像空间):", wrist_x, wrist_y, wrist_z)

            cv2.imshow('MediaPipe Hands', color_image)
            if cv2.waitKey(5) & 0xFF == 27:
                break

    finally:
        pipeline.stop()


手腕坐标 (图像空间): 367 661 1173.0
手腕坐标 (图像空间): 367 661 1177.0
手腕坐标 (图像空间): 368 661 1172.0
手腕坐标 (图像空间): 369 660 1171.0
手腕坐标 (图像空间): 368 659 1173.0
手腕坐标 (图像空间): 516 681 749.0
手腕坐标 (图像空间): 562 678 720.0
手腕坐标 (图像空间): 547 654 715.0
手腕坐标 (图像空间): 587 619 729.0
手腕坐标 (图像空间): 597 601 734.0
手腕坐标 (图像空间): 601 581 738.0
手腕坐标 (图像空间): 603 536 745.0
手腕坐标 (图像空间): 616 548 756.0
手腕坐标 (图像空间): 628 512 772.0
手腕坐标 (图像空间): 630 493 779.0
手腕坐标 (图像空间): 632 478 789.0
手腕坐标 (图像空间): 641 457 817.0
手腕坐标 (图像空间): 643 449 827.0
手腕坐标 (图像空间): 646 442 836.0
手腕坐标 (图像空间): 653 431 861.0
手腕坐标 (图像空间): 655 425 879.0
手腕坐标 (图像空间): 662 410 897.0
手腕坐标 (图像空间): 666 402 913.0
手腕坐标 (图像空间): 670 395 923.0
手腕坐标 (图像空间): 690 373 934.0
手腕坐标 (图像空间): 706 362 940.0
手腕坐标 (图像空间): 734 349 964.0
手腕坐标 (图像空间): 731 352 967.0
手腕坐标 (图像空间): 740 349 978.0
手腕坐标 (图像空间): 747 347 986.0
手腕坐标 (图像空间): 752 347 997.0
手腕坐标 (图像空间): 758 346 1025.0
手腕坐标 (图像空间): 761 346 1038.0
手腕坐标 (图像空间): 763 344 1047.0
手腕坐标 (图像空间): 764 342 1053.0
手腕坐标 (图像空间): 764 341 1056.0
手腕坐标 (图像空间): 765 3

: 

In [ ]:
# usable demo backup
import cv2
import numpy as np
import mediapipe as mp
from pyrealsense2 import pyrealsense2 as rs
from controller import Robot
from ikpy.chain import Chain

# for testing performance
import pandas as pd
original = list()
after = list()

# Mediapipe相关设置
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)  # 初始化手部检测模型

# 设置realsense相机
pipe = rs.pipeline()
config = rs.config()
# config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)
# config.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 30)

config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

profile = pipe.start(config)
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()

# 获取相机内参
intr = profile.get_stream(rs.stream.color).as_video_stream_profile().get_intrinsics()


# 定义一个模拟的UR5机器人类
class myUR5(Robot):
    def __init__(self):
        super().__init__()
        self.timestep = int(self.getBasicTimeStep())

        # 获取机器人的电机
        self.motors = [
            self.getDevice("shoulder_pan_joint"),
            self.getDevice("shoulder_lift_joint"),
            self.getDevice("elbow_joint"),
            self.getDevice("wrist_1_joint"),
            self.getDevice("wrist_2_joint"),
            self.getDevice("wrist_3_joint")
        ]

        # 定义活动链接的遮罩
        active_links_mask = [False, True, True, True, True, True, False, False]

        # 使用ikpy从URDF文件中定义UR5机器人的运动链
        self.chain = Chain.from_urdf_file("ur5e.urdf", active_links_mask=active_links_mask)

        # 获取每个关节的位置传感器
        self.position_sensors = [
            self.getDevice("shoulder_pan_joint_sensor"),
            self.getDevice("shoulder_lift_joint_sensor"),
            self.getDevice("elbow_joint_sensor"),
            self.getDevice("wrist_1_joint_sensor"),
            self.getDevice("wrist_2_joint_sensor"),
            self.getDevice("wrist_3_joint_sensor")
        ]

        # 启用位置传感器
        for sensor in self.position_sensors:
            sensor.enable(self.timestep)

    def set_pos(self, positions):
        # 将每个关节的位置设置为目标位置
        for i, pos in enumerate(positions):
            self.motors[i].setPosition(pos)
        # 步进模拟以应用新的位置
        self.step(self.timestep)

    def get_pos(self):
        # 获取每个关节的当前位置
        current_positions = [sensor.getValue() for sensor in self.position_sensors]
        return current_positions

    def set_joint_positions(self, positions):
        # 将每个关节的位置设置为目标位置
        for i, pos in enumerate(positions):
            self.motors[i].setPosition(pos)
        self.step(self.timestep)  # 步进模拟以应用新的位置

    def inverse_kinematics(self, target_position):
        # 计算目标位置的逆运动学
        joint_positions = self.chain.inverse_kinematics(target_position)
        return joint_positions[1:7]  # 忽略第一个和最后一个关节，因为它们是固定的


if __name__ == "__main__":
    arm = myUR5()  # 初始化机器人

    try:
        while True:  # 当RGB-D摄像头打开时
            frames = pipe.wait_for_frames()  # 读取帧
            color_frame = frames.get_color_frame()
            depth_frame = frames.get_depth_frame()
            if not color_frame or not depth_frame:
                continue

            # 在处理前，将BGR图像转换为RGB。
            color_image = np.asanyarray(color_frame.get_data())
            depth_image = np.asanyarray(depth_frame.get_data())
            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
            
            results = hands.process(cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB))

            if results.multi_hand_landmarks:  # 如果找到了手部标记
                for hand_landmarks in results.multi_hand_landmarks:
                    for id, lm in enumerate(hand_landmarks.landmark):
                        if id == mp_hands.HandLandmark.WRIST.value:  # 如果是手腕
                            wrist_x = int(lm.x * color_image.shape[1])
                            wrist_y = int(lm.y * color_image.shape[0])
                            wrist_z = depth_image[wrist_y, wrist_x].astype(float)
                            if wrist_z==0:
                                continue

                            original_position = [wrist_x,wrist_y,wrist_z]
                            original.append(original_position)


                            # 将像素坐标转换为真实世界坐标
                            x = (wrist_x - intr.ppx) / intr.fx * wrist_z *1.1
                            y = (wrist_y - intr.ppy) / intr.fy * wrist_z 
                            z = wrist_z * depth_scale

                            # 转换坐标系，并可能需要按比例缩放
                            scale_factor = 1  # 需要根据实际情况调整
                            target_position = [0, 0, .7]
                            after.append(target_position)
                            joint_positions = arm.inverse_kinematics(target_position)
                            arm.set_joint_positions(joint_positions)

            # 在图像上绘制手部注释
            color_image = cv2.cvtColor(color_image, cv2.COLOR_RGB2BGR)
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(color_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # 显示图像
            cv2.imshow('MediaPipe Hands', color_image)
            cv2.imshow('Depth Image', depth_colormap)
            if cv2.waitKey(5) & 0xFF == 27:  # 如果按下ESC，则退出

                outputs = pd.DataFrame({'original': original, 'after': after})
                outputs.to_csv("positions.csv",index=False)
                break

    finally:
        pipe.stop()  # 关闭RGB-D摄像头
        cv2.destroyAllWindows()  # 关闭所有窗口



In [31]:
import pandas as pd

df = pd.read_csv("positions.csv")
original = df['original']
original

0      [399, 411, 325.0]
1      [376, 397, 363.0]
2      [399, 413, 366.0]
3      [395, 387, 605.0]
4      [365, 308, 604.0]
             ...        
516    [179, 313, 473.0]
517    [151, 316, 511.0]
518    [127, 319, 577.0]
519    [133, 319, 578.0]
520     [75, 330, 611.0]
Name: original, Length: 521, dtype: object

In [32]:
import ast
b = original.to_list()
c = [ast.literal_eval(item) for item in b]


In [33]:
import numpy as np

# 假设你的3*n数组是a
a = np.array(c)

# 获取每列的最大值
max_values = np.amax(a, axis=0)
print('每列的最大值: ', max_values)

# 获取每列的最小值
min_values = np.amin(a, axis=0)
print('每列的最小值: ', min_values)


每列的最大值:  [  507.   413. 65535.]
每列的最小值:  [ 75. 271. 299.]
